<a id='home'></a>

# Many to Many Relationships in SQL


In this small project, I build a notebook which reads roster data in JSON format, parse the file, and then produce an SQLite DB that contains a User, Course, and Member table (junksion or connector table) and populate the tables from the data file.

- The DB is manipulated by using sqlite3 library.
- The JSON file is checked by queries executed directly from bash.

The workflow is as following:

1. First, the database and tables are initialized
2. Second, the roster_data JSON file is sanity checked before updating the DB
3. Third , the JSON is parsed and converted to a format which is readeble by the sqlite. Later the data is inserted to the database's appropriate tables
4. Finally, the connector table is printed with the associated tables information in order to have an idea about the data.

## Create Database and Tables

In [1]:
import sqlite3

con = sqlite3.connect('rosterdb.db')
cur = con.cursor()

cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
) ''')

con.commit()
con.close()

## Check the Raw JSON File

In [2]:
! head -20  roster_data.json

[
  [
    "Zunairah",
    "si110",
    1
  ],
  [
    "Zion",
    "si110",
    0
  ],
  [
    "Rachael",
    "si110",
    0
  ],
  [
    "Rovia",
    "si110",
    0


## Parsing JSON

In [3]:
import json

str_data = open('roster_data.json').read()

json_data = json.loads(str_data)

json_data[:10]

[[u'Zunairah', u'si110', 1],
 [u'Zion', u'si110', 0],
 [u'Rachael', u'si110', 0],
 [u'Rovia', u'si110', 0],
 [u'Folarinwa', u'si110', 0],
 [u'Kyna', u'si110', 0],
 [u'Anayah', u'si110', 0],
 [u'Kierin', u'si110', 0],
 [u'Abel', u'si110', 0],
 [u'Yana', u'si110', 0]]

## Insert values to the tables

In [4]:
con = sqlite3.connect('rosterdb.db')
cur = con.cursor()

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2]

    cur.execute('''INSERT OR IGNORE INTO User (name) 
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title) 
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id,role) VALUES ( ?, ?, ?  )''', 
        ( user_id, course_id,role  ) )

    con.commit()


#### The Connector Table

In [5]:
query = '''SELECT User.name, Member.role, Course.title  
    FROM User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY Course.title,Member.role,User.name '''

with sqlite3.connect('rosterdb.db') as con:
    cur = con.cursor()
    cur.execute(query)
    lst = cur.fetchall()
    print "%10s %6s %9s" %('Name', 'Role', 'Title')
    print '---------------------------------'
    for element in lst[:20]:
        print "%10s %5s %10s" %element
    

      Name   Role     Title
---------------------------------
     Aaima     0      si106
     Bowen     0      si106
     Cally     0      si106
     Darby     0      si106
     Idris     0      si106
      Inez     0      si106
   Issiaka     0      si106
  Izabella     0      si106
     Jamal     0      si106
    Kelsie     0      si106
   Leechan     0      si106
     Manal     0      si106
     Matia     0      si106
  Mercedez     0      si106
  Mohammad     0      si106
Pardeepraj     0      si106
      Skie     0      si106
  Stewarty     0      si106
     Tarik     0      si106
   Zakiyya     0      si106


[Home](#home)